In [9]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import names
import sklearn

### Training a Naive Bayes Classifier to classify gender of names

In [10]:
def gender_features(word):
    return {'suffix1': word[-1:],
            'suffix2': word[-2:]}

In [11]:
import random
labeled_names = ([(name, 'male') for name in names.words('male.txt')]+
             [(name, 'female') for name in names.words('female.txt')])
random.shuffle(labeled_names)

LookupError: 
**********************************************************************
  Resource [93mnames[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('names')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/names[0m

  Searched in:
    - '/Users/karenwang/nltk_data'
    - '/Users/karenwang/opt/anaconda3/nltk_data'
    - '/Users/karenwang/opt/anaconda3/share/nltk_data'
    - '/Users/karenwang/opt/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
featuresets = [(gender_features(n), gender) 
               for (n, gender) in labeled_names]

classifier = nltk.NaiveBayesClassifier.train(featuresets)

In [ ]:
#test on a new name
classifier.classify(gender_features('Trinity'))

### Use Classifier to Classify Names in Credits Dataset

In [ ]:
credits = pd.read_csv('data/credits.csv')
credits.head()

In [ ]:
# credits['gender'] = classifier.classify(gender_features(credits['name'].split()[0]))
credits['gender'] = credits['name'].apply(lambda x : classifier.classify(gender_features(x.split()[0])))

In [4]:
credits.head(10)

AttributeError: '_Printer' object has no attribute 'head'

### Use Classifier on titles_and_credits

In [12]:
titles_and_credits = pd.read_csv('data/titles_and_credits.csv')
titles_and_credits.head()

,Unnamed: 0.1,Unnamed: 0,id,title,type,description,release_year,age_certification,runtime,genres,...,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,person_id,name,character,role
0,0,1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",...,NaN,tt0075314,8.2,808582.0,40.965,8.179,3748,Robert De Niro,Travis Bickle,ACTOR
1,1,1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",...,NaN,tt0075314,8.2,808582.0,40.965,8.179,14658,Jodie Foster,Iris Steensma,ACTOR
2,2,1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",...,NaN,tt0075314,8.2,808582.0,40.965,8.179,7064,Albert Brooks,Tom,ACTOR
3,3,1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",...,NaN,tt0075314,8.2,808582.0,40.965,8.179,3739,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,4,1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",...,NaN,tt0075314,8.2,808582.0,40.965,8.179,48933,Cybill Shepherd,Betsy,ACTOR


In [13]:
titles_and_credits['gender'] = titles_and_credits['name'].apply(
    lambda x : classifier.classify(gender_features(x.split()[0])))

NameError: name 'classifier' is not defined

In [14]:
titles_and_credits.head()

,Unnamed: 0.1,Unnamed: 0,id,title,type,description,release_year,age_certification,runtime,genres,...,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,person_id,name,character,role
0,0,1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",...,NaN,tt0075314,8.2,808582.0,40.965,8.179,3748,Robert De Niro,Travis Bickle,ACTOR
1,1,1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",...,NaN,tt0075314,8.2,808582.0,40.965,8.179,14658,Jodie Foster,Iris Steensma,ACTOR
2,2,1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",...,NaN,tt0075314,8.2,808582.0,40.965,8.179,7064,Albert Brooks,Tom,ACTOR
3,3,1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",...,NaN,tt0075314,8.2,808582.0,40.965,8.179,3739,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,4,1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",...,NaN,tt0075314,8.2,808582.0,40.965,8.179,48933,Cybill Shepherd,Betsy,ACTOR


### Gender Proportion vs Popularity of Titles

In [12]:
#groupby title and calculate proportion of female names - by doing onehotencoding of gender, then take avg of male and female
#for each title, create a separate observation for every genre that is listed
#scatterplot of %female vs popularity

In [15]:
def prop_female(x):
    gender_count = x['gender'].value_counts()
    if (len(gender_count) == 2): #if there are females (all observations either have both M/F or only M names)
        return gender_count[1] / (gender_count[0] + gender_count[1])
    else: 
        return 0

df_prop_female = titles_and_credits.groupby('id').apply(prop_female).to_frame(name = 'prop_female')

KeyError: 'gender'

In [17]:
titles = pd.read_csv('data/titles_clean.csv')

top_genres_mask = titles['genres'].apply(lambda x: ('drama' in x)  or ('comedy' in x) or ('thriller' in x) or ('action' in x) or ('romance' in x) or ('documentation' in x) or ('crime' in x))
titles = titles[top_genres_mask]

print(titles.shape)
titles.head()

(2509, 16)


,Unnamed: 0,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


In [20]:
titles = pd.merge(titles, df_prop_female, on = 'id')
print(titles.shape)
titles.head()

NameError: name 'df_prop_female' is not defined

Making a dataframe for titles that include each of the 7 genres, and calculating descriptive stats for their prop_female and imdb_popularity columns.

In [38]:
# df of shows or movies that included 'drama' as a genre label
includes_drama_mask = titles['genres'].apply(lambda x: ('drama' in x))
includes_drama = titles[includes_drama_mask] 
print(includes_drama.shape)
# print(includes_drama.columns.values)
# print(includes_drama[['tmdb_popularity', 'prop_female']])
includes_drama.head()

includes_drama[['tmdb_popularity', 'prop_female']].describe()
includes_drama[['tmdb_popularity', 'prop_female']].median()


 # df of shows or movies that included 'comedy' as a genre label
includes_comedy_mask = titles['genres'].apply(lambda x: ('comedy' in x))
includes_comedy = titles[includes_comedy_mask]
print(includes_comedy.shape)
# print(includes_comedy.columns.values)
# print(includes_comedy[['tmdb_popularity', 'prop_female']])
includes_comedy.head()

includes_comedy[['tmdb_popularity', 'prop_female']].describe()
includes_comedy[['tmdb_popularity', 'prop_female']].median()


# df of shows or movies that included 'thriller' as a genre label
includes_thriller_mask = titles['genres'].apply(lambda x: ('thriller' in x))
includes_thriller = titles[includes_thriller_mask]
print(includes_thriller.shape)
# print(includes_thriller.columns.values)
# print(includes_thriller[['tmdb_popularity', 'prop_female']])
includes_thriller.head()

includes_thriller[['tmdb_popularity', 'prop_female']].describe()
includes_thriller[['tmdb_popularity', 'prop_female']].median()


# df of shows or movies that included 'action' as a genre label
includes_action_mask = titles['genres'].apply(lambda x: ('action' in x))
includes_action = titles[includes_action_mask]
print(includes_action.shape)
# print(includes_action.columns.values)
# print(includes_action[['tmdb_popularity', 'prop_female']])
includes_action.head()

includes_action[['tmdb_popularity', 'prop_female']].describe()
includes_action[['tmdb_popularity', 'prop_female']].median()


# df of shows or movies that included 'romance' as a genre label
includes_romance_mask = titles['genres'].apply(lambda x: ('romance' in x))
includes_romance = titles[includes_romance_mask]
print(includes_romance.shape)
# print(includes_romance.columns.values)
# print(includes_romance[['tmdb_popularity', 'prop_female']])
includes_romance.head()

includes_romance[['tmdb_popularity', 'prop_female']].describe()
includes_romance[['tmdb_popularity', 'prop_female']].median()


# df of shows or movies that included 'documentation' as a genre label
includes_documentation_mask = titles['genres'].apply(lambda x: ('documentation' in x))
includes_documentation = titles[includes_documentation_mask]
print(includes_documentation.shape)
# print(includes_documentation.columns.values)
# print(includes_documentation[['tmdb_popularity', 'prop_female']])
includes_documentation.head()

includes_documentation[['tmdb_popularity', 'prop_female']].describe()
includes_documentation[['tmdb_popularity', 'prop_female']].median()


# df of shows or movies that included 'crime' as a genre label
includes_crime_mask = titles['genres'].apply(lambda x: ('crime' in x))
includes_crime = titles[includes_crime_mask]
print(includes_crime.shape)
# print(includes_crime.columns.values)
# print(includes_crime[['tmdb_popularity', 'prop_female']])
includes_crime.head()

includes_crime[['tmdb_popularity', 'prop_female']].describe()
includes_crime[['tmdb_popularity', 'prop_female']].median()

(1048, 16)


KeyError: "['prop_female'] not in index"